In [2]:
import pandas as pd
import numpy as np
from custom_xgoost.xgboost_genetic_algorithm_split import CustomXGBoost
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

df = pd.read_csv("data/ames.csv")
df = df.apply(pd.to_numeric, errors='coerce').dropna(axis=1, how='all')

y = df["SalePrice"].values
X = df.drop(columns=["SalePrice"]).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = CustomXGBoost(n_estimators=10, learning_rate=0.1, max_depth=3, min_samples_split=5)
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_mse = np.mean((y_train - y_train_pred) ** 2)
test_mse = np.mean((y_test - y_test_pred) ** 2)
train_rmse = np.sqrt(train_mse)
test_rmse = np.sqrt(test_mse)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("Train MSE:", train_mse)
print("Test MSE:", test_mse)
print()
print("Train RMSE:", train_rmse)
print("Test RMSE:", test_rmse)
print()
print("Train R²:", train_r2)
print("Test R²:", test_r2)


Train MSE: 34898788191.727554
Test MSE: 35717702425.335014

Train RMSE: 186812.17356405754
Test RMSE: 188991.27605615824

Train R²: -4.462680145236739
Test R²: -4.889803411734647
